In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

orders = pd.read_csv(r'C:\Users\admin\OneDrive\Desktop\innomatics_intern\data\orders.csv')
print(orders.head())
print(orders.info())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total

In [7]:
users = pd.read_json(r'C:\Users\admin\OneDrive\Desktop\innomatics_intern\data\users.json')
print(users.head())
print(users.info())

   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB
None


In [ ]:
import sqlite3

restaurants_path = r'C:\Users\admin\OneDrive\Desktop\innomatics_intern\data\restaurants.sql'

conn = sqlite3.connect("food_delivery.db")
cursor = conn.cursor()

# To solve problem of multiple table reads i.e. to clean the state
cursor.execute("DROP TABLE IF EXISTS restaurants;")

with open(restaurants_path, 'r', encoding='utf-8') as f:
    restaurants_path = f.read()

cursor.executescript(restaurants_path)

conn.commit()
conn.close()


In [35]:
conn = sqlite3.connect(r'C:\Users\admin\OneDrive\Desktop\innomatics_intern\food_delivery.db')

restaurants_df = pd.read_sql_query(
    "SELECT * FROM restaurants;",
    conn
)

conn.close()

print(restaurants_df.head())

   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [18]:
order_and_users = pd.merge(
    orders,
    users,
    on = "user_id",
    how = "left"
)

In [21]:
final_df = pd.merge(
    order_and_users,
    restaurants_df,
    on = "restaurant_id",
    how = "left"
)

In [32]:
# To find if any null values

final_df[["user_id", "restaurant_id" ]].isnull().sum

<bound method DataFrame.sum of       user_id  restaurant_id
0       False          False
1       False          False
2       False          False
3       False          False
4       False          False
...       ...            ...
9995    False          False
9996    False          False
9997    False          False
9998    False          False
9999    False          False

[10000 rows x 2 columns]>

In [ ]:
#Checking for any missed values

print(len(order_and_users))
print(len(final_df))

10000
10000


In [31]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])

# Monthly order
final_df['order_month'] = final_df['order_date'].dt.to_period("M")
final_df['order_date'].dtype

dtype('<M8[ns]')

In [25]:
# Explorting the final dataset

final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index="False"
)

In [29]:
# Final check

print("Number of Orders : ", len(orders))
print("Number of rows in Final dataset : ", len(final_df))
assert len(final_df) == len(orders)

Number of Orders :  10000
Number of rows in Final dataset :  10000
